In [1]:
# 라이브러리를 불러옵니다.
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib.colors import LogNorm
from matplotlib import rc
import seaborn as sns

import networkx as nx

#### 폰트 설정

In [2]:
# 한글 폰트 설정
font_path = '/usr/share/fonts/truetype/nanum/MalgunGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=18)
rc('font', family='Malgun Gothic')

#### 그래프 생성용 함수

In [3]:
def create_graph(data):
    G = nx.Graph()
    
    # 노드를 추가합니다.
    for atc_code in data['ATC_code'].unique():
        G.add_node(atc_code)
        
    # user_id와 pres_id로 그룹화하여 동시 발생을 찾습니다.
    user_cooccurrences = {}
    for user_id, user_group in data.groupby('user_id'):
        # 각 user_id에 대해 co-occurrence를 집합으로 저장합니다.
        cooccurrences = set()
        for pres_id, pres_group in user_group.groupby('pres_id'):
            atc_codes = pres_group['ATC_code'].unique()
            for i in range(len(atc_codes)):
                for j in range(i + 1, len(atc_codes)):
                    cooccurrence = tuple(sorted((atc_codes[i], atc_codes[j])))
                    cooccurrences.add(cooccurrence)
        user_cooccurrences[user_id] = cooccurrences

    # co-occurrence를 그래프에 추가합니다.
    for cooccurrences in user_cooccurrences.values():
        for atc_code1, atc_code2 in cooccurrences:
            if G.has_edge(atc_code1, atc_code2):
                G[atc_code1][atc_code2]['weight'] += 1
            else:
                G.add_edge(atc_code1, atc_code2, weight=1)

    return G

#### 만성질환 목록 및 매핑

In [46]:
# 만성질환 목록을 불러옵니다.
comor_mapping = pd.read_csv('data/mapping/comorbidities_mapping.csv', encoding='utf-8')

# ATC 코드를 약품명으로 바꿔줍니다.
atc_to_name = comor_mapping.set_index('ATC코드')['주성분명'].to_dict()

# 만성질환 질병 종류를 구분해줍니다. 
class_map = comor_mapping[['ATC코드', 'comorbidity_class']].drop_duplicates(subset=['ATC코드'], keep='first')
atc_to_class = class_map.set_index('ATC코드')['comorbidity_class'].to_dict()

# 만성질환 질병 종류를 구분해줍니다. (여러 질병이 하나의 ATC 코드에 매핑되는 경우)
comor_mapping['comorbidity_classes'] = comor_mapping.groupby('ATC코드')['comorbidity_class'].transform(lambda x: ' '.join(x.unique()))
atc_to_classes = comor_mapping.set_index('ATC코드')['comorbidity_classes'].to_dict()

In [36]:
temp = comor_mapping.groupby(['ATC코드'])['comorbidity_class'].nunique()
temp = temp[temp > 1]
temp

ATC코드
C03AA03    2
C03CA      2
C03CA01    2
C03CA04    2
C07AB02    2
          ..
R03DA04    2
R03DA05    2
R03DC01    2
R03DC03    2
R03DX05    2
Name: comorbidity_class, Length: 71, dtype: int64

#### 컬럼명 변경용 딕셔너리

In [5]:
# 컬럼 이름들을 영문으로 바꿔줍니다
cols_rename = {
    '기준년도': 'year',
    '가입자일련번호': 'user_id',
    '처방내역일련번호': 'pres_id',
    '일련번호': 'med_id',
    '성별코드': 'gender',
    '연령대코드(5세단위)': 'age_group',
    '시도코드': 'region',
    '요양개시일자': 'start_date',
    '약품일반성분명코드': 'mi_code',
    '1회 투약량': 'dose_per_time',
    '1일투약량': 'dose_per_day',
    '총투여일수': 'total_days',
    '단가': 'unit_price',
    '금액': 'total_price',
    '데이터 공개일자': 'data_release_date',
    'ATC코드': 'ATC_code'
}

## 65세 이상 항우울제 미처방 환자 약물 네트워크 분석

In [47]:
# 데이터를 불러옵니다.
data_senior = pd.read_csv('./data/prescription_over_65_comor_sample_2021.csv')

In [48]:
# 데이터의 첫 다섯 줄을 출력합니다.
print(data_senior.shape)
data_senior.head()

(201593, 16)


,기준년도,가입자일련번호,처방내역일련번호,일련번호,성별코드,연령대코드(5세단위),시도코드,요양개시일자,약품일반성분명코드,1회 투약량,1일투약량,총투여일수,단가,금액,데이터 공개일자,ATC코드
0,2021,403017,154510,1,2,14,41,2021-12-21,111001ATE,1.0,1,30,77.0,2310,2022-08-11,B01AC06
1,2021,547179,238048,1,2,16,41,2021-05-26,111001ATE,1.0,1,30,77.0,2310,2022-08-11,B01AC06
2,2021,287268,320180,1,1,15,41,2021-11-12,111001ATE,1.0,1,30,77.0,2310,2022-08-11,B01AC06
3,2021,403017,320961,1,2,14,41,2021-05-21,111001ATE,1.0,1,30,77.0,2310,2022-08-11,B01AC06
4,2021,547179,461888,1,2,16,41,2021-10-26,111001ATE,1.0,1,30,77.0,2310,2022-08-11,B01AC06


---

In [49]:
df_senior = data_senior.rename(columns=cols_rename)

In [50]:
# 한 환자당 처방전의 개수를 세어봅니다.
print(f"고려되는 총 환자 수: {df_senior['user_id'].nunique()}")
df_senior.groupby('user_id')['pres_id'].count().describe()

고려되는 총 환자 수: 12164


count    12164.000000
mean        16.572920
std         15.689849
min          1.000000
25%          5.000000
50%         12.000000
75%         23.000000
max        175.000000
Name: pres_id, dtype: float64

### 65세 이상 그래프 생성

In [51]:
# 그래프를 생성합니다.
G_senior = create_graph(df_senior)

# 그래프의 노드와 엣지 수를 출력합니다.
print(f"Number of nodes: {G_senior.number_of_nodes()}")
print(f"Number of edges: {G_senior.number_of_edges()}")

Number of nodes: 197
Number of edges: 4672


In [52]:
# 그래프에 label을 추가합니다.
for node in G_senior.nodes:
    G_senior.nodes[node]['label'] = atc_to_name[node]
    G_senior.nodes[node]['comorbidity'] = atc_to_class[node]
    G_senior.nodes[node]['comorbidities'] = atc_to_classes[node]

In [53]:
# Gephi로 시각화를 위해 그래프를 저장합니다.
nx.write_gexf(G_senior, './data/gephi/no_depression_65_prescription_network.gexf')

In [57]:
comor_mapping['comorbidity_classes'].unique()

array(['COPD_med asthma_med', 'COPD_med', 'diabetes_med',
       'high_bp_med stroke_med', 'high_bp_med',
       'dyslipidemia_med stroke_med', 'dyslipidemia_med', 'dementia_med',
       'stroke_med', 'asthma_med', 'antineoplastic'], dtype=object)

Color Palette Summary:

| Medication          | Color Name                | HEX Code | RGB             |
|---------------------|---------------------------|----------|-----------------|
| COPD_med            | Pastel Red                | #FFB3BA  | (255, 179, 186) |
| diabetes_med        | Pastel Orange             | #FFDFBA  | (255, 223, 186) |
| high_bp_med         | Pastel Yellow             | #FFFFBA  | (255, 255, 186) |
| dyslipidemia_med    | Pastel Green              | #BAFFC9  | (186, 255, 201) |
| dementia_med        | Pastel Blue               | #BAE1FF  | (186, 225, 255) |
| stroke_med          | Pastel Purple             | #E3BAFF  | (227, 186, 255) |
| asthma_med          | Pastel Pink               | #FFB3FF  | (255, 179, 255) |
| antineoplastic      | Pastel Brown              | #F3E5AB  | (243, 229, 171) |
| COPD_med asthma_med | Blended Pastel Red-Pink   | #FFB3DD  | (255, 179, 221) |
| high_bp_med stroke_med | Blended Pastel Yellow-Purple | #F1DDDD  | (241, 221, 221) |
| dyslipidemia_med stroke_med | Blended Pastel Green-Purple | #CFDDE4  | (207, 221, 228) |

| Medication                     | Color Name          | HEX Code | RGB             |
|--------------------------------|---------------------|----------|-----------------|
| high_bp_med                    | Vibrant Yellow      | #FFDD00  | (255, 221, 0)   |
| high_bp_med / stroke_med       | Vibrant Gold        | #CC9933  | (204, 153, 51)  |
| stroke_med                     | Bright Purple       | #9933FF  | (153, 51, 255)  |
| dyslipidemia_med / stroke_med  | Emerald Purple      | #669966  | (102, 153, 102) |
| dyslipidemia_med               | Vibrant Green       | #00CC66  | (0, 204, 102)   |
| COPD_med                       | Bright Red          | #FF3366  | (255, 51, 102)  |
| COPD_med / asthma_med          | Rosy Magenta        | #FF6699  | (255, 102, 153) |
| asthma_med                     | Bright Pink         | #FF66FF  | (255, 102, 255) |
| antineoplastic                 | Warm Brown          | #996633  | (153, 102, 51)  |
| diabetes_med                   | Vibrant Orange      | #FF9933  | (255, 153, 51)  |
| dementia_med                   | Soft Blue           | #3399FF  | (51, 153, 255)  |

---
## 10대 항우울제 미처방 환자 약물 네트워크 분석

In [13]:
# 데이터를 불러옵니다.
data_teens = pd.read_csv('./data/prescription_teens_comor_sample_2021.csv')

In [14]:
# 데이터의 첫 다섯 줄을 출력합니다.
print(data_teens.shape)
data_teens.head()

(680, 16)


,기준년도,가입자일련번호,처방내역일련번호,일련번호,성별코드,연령대코드(5세단위),시도코드,요양개시일자,약품일반성분명코드,1회 투약량,1일투약량,총투여일수,단가,금액,데이터 공개일자,ATC코드
0,2021,622439,2382371,1,2,4,41,2021-03-12,117903ATB,1.0,1,10,132.0,1320,2022-08-11,C07AB07
1,2021,622439,582385,1,2,4,41,2021-10-15,117902ATB,1.0,1,30,222.0,6660,2022-08-11,C07AB07
2,2021,70276,1471269,1,1,3,41,2021-11-29,374601ATB,1.0,1,3,774.0,2322,2022-08-11,R03DC03
3,2021,70276,1473024,1,1,3,41,2021-11-26,374601ATB,1.0,1,3,774.0,2322,2022-08-11,R03DC03
4,2021,293925,2458457,1,2,4,41,2021-06-09,636700CSI,1.0,1,1,31915.0,31915,2022-08-11,R03AK10


In [15]:
# 컬럼 이름들을 영문으로 바꿔줍니다
df_teens = data_teens.rename(columns=cols_rename)

In [16]:
# 한 환자당 처방전의 개수를 세어봅니다.
print(f"고려되는 총 환자 수: {df_teens['user_id'].nunique()}")
df_teens.groupby('user_id')['pres_id'].count().describe()

고려되는 총 환자 수: 246


count    246.000000
mean       2.764228
std        2.762966
min        1.000000
25%        1.000000
50%        2.000000
75%        3.750000
max       17.000000
Name: pres_id, dtype: float64

---
### 십대 그래프 생성

In [17]:
# 그래프를 생성합니다.
G_teens = create_graph(df_teens)

# 그래프의 노드와 엣지 수를 출력합니다.
print(f"Number of nodes: {G_teens.number_of_nodes()}")
print(f"Number of edges: {G_teens.number_of_edges()}")

Number of nodes: 28
Number of edges: 35


In [18]:
# 그래프에 label을 추가합니다.
for node in G_teens.nodes:
    G_teens.nodes[node]['label'] = atc_to_name[node]
    G_teens.nodes[node]['comorbidity'] = atc_to_class[node]

# Gephi로 시각화를 위해 그래프를 저장합니다.
nx.write_gexf(G_teens, './data/gephi/no_depression_teens_prescription_network.gexf')

---
## 중년 항우울제 미처방 환자 약물 네트워크 분석

In [19]:
# 데이터를 불러옵니다.
data_middle_aged = pd.read_csv('./data/prescription_middle_aged_comor_sample_2021.csv')

In [20]:
# 데이터의 첫 다섯 줄을 출력합니다.
print(data_middle_aged.shape)
data_middle_aged.head()

(157062, 16)


,기준년도,가입자일련번호,처방내역일련번호,일련번호,성별코드,연령대코드(5세단위),시도코드,요양개시일자,약품일반성분명코드,1회 투약량,1일투약량,총투여일수,단가,금액,데이터 공개일자,ATC코드
0,2021,182436,516227,1,1,12,41,2021-03-13,111001ATE,1.0,1,30,77.0,2310,2022-08-11,B01AC06
1,2021,182436,627161,1,1,12,41,2021-06-14,111001ATE,1.0,1,30,77.0,2310,2022-08-11,B01AC06
2,2021,182436,1961801,1,1,12,41,2021-04-12,111001ATE,1.0,1,30,77.0,2310,2022-08-11,B01AC06
3,2021,182436,3443548,1,1,12,41,2021-02-09,111001ATE,1.0,1,30,77.0,2310,2022-08-11,B01AC06
4,2021,733801,412966,1,1,11,41,2021-11-17,111001ATE,1.0,1,30,77.0,2310,2022-08-11,B01AC06


In [21]:
# 컬럼 이름들을 영문으로 바꿔줍니다
df_middle_aged = data_middle_aged.rename(columns=cols_rename)

In [22]:
# 한 환자당 처방전의 개수를 세어봅니다.
print(f"고려되는 총 환자 수: {df_middle_aged['user_id'].nunique()}")
df_middle_aged.groupby('user_id')['pres_id'].count().describe()

고려되는 총 환자 수: 21110


count    21110.000000
mean         7.440171
std          9.542537
min          1.000000
25%          2.000000
50%          4.000000
75%         10.000000
max        133.000000
Name: pres_id, dtype: float64

### 중년층 그래프 생성

In [23]:
# 그래프를 생성합니다.
G_middle_aged = create_graph(df_middle_aged)

# 그래프의 노드와 엣지 수를 출력합니다.
print(f"Number of nodes: {G_middle_aged.number_of_nodes()}")
print(f"Number of edges: {G_middle_aged.number_of_edges()}")

Number of nodes: 190
Number of edges: 3116


In [24]:
# 그래프에 label을 추가합니다.
for node in G_middle_aged.nodes:
    G_middle_aged.nodes[node]['label'] = atc_to_name[node]
    G_middle_aged.nodes[node]['comorbidity'] = atc_to_class[node]

# Gephi로 시각화를 위해 그래프를 저장합니다.
nx.write_gexf(G_middle_aged, './data/gephi/no_depression_middle_aged_prescription_network.gexf')

---
## 20~30대 항우울제 미처방 환자 약물 네트워크 분석

In [25]:
# 데이터를 불러옵니다.
data_young_adults = pd.read_csv('./data/prescription_young_adults_comor_sample_2021.csv')

In [26]:
# 데이터의 첫 다섯 줄을 출력합니다.
print(data_young_adults.shape)
data_young_adults.head()

(67848, 16)


,기준년도,가입자일련번호,처방내역일련번호,일련번호,성별코드,연령대코드(5세단위),시도코드,요양개시일자,약품일반성분명코드,1회 투약량,1일투약량,총투여일수,단가,금액,데이터 공개일자,ATC코드
0,2021,323015,659057,1,1,7,41,2021-09-10,111001ATE,1.0,1,30,77.0,2310,2022-08-11,B01AC06
1,2021,323015,1201902,1,1,7,41,2021-08-07,111001ATE,1.0,1,30,77.0,2310,2022-08-11,B01AC06
2,2021,323015,3653408,1,1,7,41,2021-07-10,111001ATE,1.0,1,30,77.0,2310,2022-08-11,B01AC06
3,2021,624411,132193,1,2,8,41,2021-07-12,191501ATB,1.0,1,30,49.0,1470,2022-08-11,A10BA02
4,2021,286759,844417,1,1,8,41,2021-03-15,191502ATB,1.0,1,30,70.0,2100,2022-08-11,A10BA02


In [27]:
# 컬럼 이름들을 영문으로 바꿔줍니다
df_young_adults = data_young_adults.rename(columns=cols_rename)

In [28]:
# 한 환자당 처방전의 개수를 세어봅니다.
print(f"고려되는 총 환자 수: {df_young_adults['user_id'].nunique()}")
df_young_adults.groupby('user_id')['pres_id'].count().describe()

고려되는 총 환자 수: 21110


count    21110.000000
mean         3.214022
std          5.069917
min          1.000000
25%          1.000000
50%          2.000000
75%          3.000000
max        332.000000
Name: pres_id, dtype: float64

### 20~30대 그래프 생성

In [29]:
# 그래프를 생성합니다.
G_young_adults = create_graph(df_young_adults)

# 그래프의 노드와 엣지 수를 출력합니다.
print(f"Number of nodes: {G_young_adults.number_of_nodes()}")
print(f"Number of edges: {G_young_adults.number_of_edges()}")

Number of nodes: 155
Number of edges: 1278


In [30]:
# 그래프에 label을 추가합니다.
for node in G_young_adults.nodes:
    G_young_adults.nodes[node]['label'] = atc_to_name[node]
    G_young_adults.nodes[node]['comorbidity'] = atc_to_class[node]

In [31]:
# Gephi로 시각화를 위해 그래프를 저장합니다.
nx.write_gexf(G_young_adults, './data/gephi/no_depression_young_adults_prescription_network.gexf')